In [1]:
import os
from utils import mkdirs
import image as img
from resnet_model import Resnet3DBuilder
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from HistoryGraph import HistoryGraph
from math import ceil

Using TensorFlow backend.


In [3]:
target_size = (64,96)
nb_frames = 16
skip = 1
nb_classes = 27
batch_size = 64
input_shape = (nb_frames, ) + target_size + (3,)

In [4]:
workers = 8
use_multiprocessing = False
max_queue_size = 20

In [5]:
import pandas as pd
import numpy as np

class DataLoader():
    def __init__(self, path_vid, path_labels, path_train=None, path_val=None, path_test=None):
        self.path_vid = path_vid
        self.path_labels = path_labels
        self.path_train = path_train
        self.path_val = path_val
        self.path_test = path_test
        self.get_labels(path_labels)
        
        if self.path_train:
            self.train_df = self.load_video_labels(self.path_train)
        if self.path_val:
            self.val_df = self.load_video_labels(self.path_val)
        if self.path_test:
            self.test_df = self.load_video_labels(self.path_test, mode="input")
    
    
    def get_labels(self, path_labels):
        self.labels_df = pd.read_csv(path_labels, names=['label'])
        #extract labels from dataframe
        self.labels = [str(label[0]) for label in self.labels_df.values]
        #print(self.labels)
        self.n_labels = len(self.labels)
        #print(self.n_labels)
        #create dictionaries to convert label to int and backwards
        self.label_to_int = dict(zip(self.labels, range(self.n_labels)))
        #print(self.label_to_int)
        self.int_to_labels = dict(enumerate(self.labels))
        #print(self.int_to_labels)

        




    def load_video_labels(self, path_subset, mode="label"):
        if mode == "input":
            # For test.csv, which has only video IDs
            df = pd.read_csv(path_subset, names=["video_id"])
        elif mode == "label":
            # For train.csv, validation.csv, and test-answers.csv, which have video IDs and labels
            df = pd.read_csv(path_subset, sep=';', names=["video_id", "label"])
            df = df[df.label.isin(self.labels)]
        
        #print(df)
        return df

In [6]:
data_root = r"C:\Users\Dawud Iqbal\Documents\Final Project\Final Project\Implementation 2\Dataset"
csv_labels = r"C:\Users\Dawud Iqbal\Documents\Final Project\Final Project\Implementation 2\Dataset\labels.csv"
csv_train = r"C:\Users\Dawud Iqbal\Documents\Final Project\Final Project\Implementation 2\Dataset\train.csv"
csv_val = r"C:\Users\Dawud Iqbal\Documents\Final Project\Final Project\Implementation 2\Dataset\validation.csv"
csv_test = r"C:\Users\Dawud Iqbal\Documents\Final Project\Final Project\Implementation 2\Dataset\test.csv"
data_vid = r"C:\Users\Dawud Iqbal\Documents\Final Project\Final Project\Implementation 2\Dataset\videos"
model_name = 'resnet_3d_model'
data_model = r"C:\Users\Dawud Iqbal\Documents\Final Project\Final Project\Implementation 2\Dataset\model"

In [7]:
path_model = os.path.join(data_root, data_model, model_name)
path_vid = os.path.join(data_root, data_vid)
path_labels = os.path.join(data_root, csv_labels)
path_train = os.path.join(data_root, csv_train)
path_val = os.path.join(data_root, csv_val)
path_test = os.path.join(data_root, csv_test)

In [8]:
data = DataLoader(path_vid, path_labels, path_train, path_val)
mkdirs(path_model, 0o755)
mkdirs(os.path.join(path_model, "graphs"), 0o755)

In [9]:
gen = img.ImageDataGenerator()
gen_train = gen.flow_video_from_dataframe(data.train_df, path_vid, path_classes = path_labels, x_col='video_id', y_col="label", target_size=target_size, batch_size=batch_size,nb_frames = nb_frames, skip = skip, has_ext = True)
gen_val = gen.flow_video_from_dataframe(data.val_df, path_vid, path_classes = path_labels, x_col='video_id', y_col="label", target_size=target_size, batch_size=batch_size,nb_frames = nb_frames, skip = skip, has_ext = True)

Found 118562 video folders belonging to 27 classes.
Found 14787 video folders belonging to 27 classes.


In [10]:
resnet_model = Resnet3DBuilder.build_resnet_101(input_shape, nb_classes, drop_rate = 0.5)
optimizer = SGD(lr=0.01, momentum = 0.9, decay = 0.0001, nesterov=False)
resnet_model.compile(optimizer = optimizer, loss="categorical_crossentropy" , metrics=["accuracy"])
model_file = os.path.join(path_model, 'resnetmodel.hdf5')

In [11]:
model_checkpointer = ModelCheckpoint(model_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [12]:
history_graph = HistoryGraph(model_path_name = os.path.join(path_model, "graphs"))

In [13]:
nb_sample_train = data. train_df["video_id"].size
nb_sample_val = data.val_df["video_id"].size

In [ ]:
resnet_model.fit_generator(
    generator = gen_train,
    steps_per_epoch=ceil(nb_sample_train/batch_size),
    epochs=30,
    validation_data=gen_val,
    validation_steps=30,
    shuffle=True,
    verbose=1,
    workers=workers,
    max_queue_size = max_queue_size,
    use_multiprocessing = use_multiprocessing,
    callbacks = [model_checkpointer, history_graph])